# SABAT SMFI

- Estimates detection efficiency for Fuchsin in new laser setup TOPATU

In [2]:
%matplotlib inline
from IPython.display import Image
from IPython.display import Math
from IPython.display import Latex

In [3]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time
import warnings
import datetime 
print("Last updated on ", time.asctime())

Last updated on  Mon Jun 22 19:19:09 2020


In [4]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import tables            as tb
import random
import glob
import warnings
sns.set()
import matplotlib.pyplot as plt


In [5]:
from dataclasses import dataclass

In [6]:
from  invisible_cities.core.system_of_units import *
from  invisible_cities.core.system_of_units import MHZ

In [7]:
from sabat.sbt_types import FoV, CircularFoV
from sabat.sbt_types import Laser, PulsedLaser, GLaser, GaussianBeam
from sabat.sbt_types import Molecule
from sabat.sbt_types import Molecule2P
from sabat.sbt_types import DyeSample
from sabat.sbt_types import Microscope
from sabat.sbt_types import CCD

In [8]:
from sabat.sbt_core import  power_density
from sabat.sbt_core import  photon_density
from sabat.sbt_core import  fluorescence_per_molecule
from sabat.sbt_core import  duration_of_fluorescence
from sabat.sbt_core import  diffraction_limit
from sabat.sbt_core import  photoelectrons_per_pixel
# from sabat.sbt_core import  fluorescence_2p
from sabat.sbt_core import  absorbed_photons_per_fluorophore_per_pulse_2p

# SABAT SMFI

## Authors

- J.J. Gomez-Cadenas 

- P. Herrero Gómez

[NEXT](https://next.ific.uv.es/next/)

#### Contact


- [Email](mailto:jjgomezcadenas@dipc.org)




## Diode laser
Beam size = ø 3.5 mm

In [9]:
weakLaser = Laser(lamda=532 * nm, power = 4.5 * milliwatt)
weakLaser


        Laser:
        wavelength                =5.3e+02 nm
        photon energy             =2.3e+00 eV
        power                     =4.5e+00 mW
        energy per second         =4.5e+00 mJ
        photons per second        =1.2e+16 ph/second
        

## Microscope
Dichroic cuts at 567 nm, with transmission efficiency of ~95%

https://www.thorlabs.com/newgrouppage9.cfm?objectgroup_id=3313&pn=DMLP567R

No filter, so eff_filter = 1

OB has NA = 0.3, M = 30x


In [10]:
mmc = Microscope(name='TOPATU', numerical_aperture=0.3, magnification=30, eff_filter=1, eff_dichroic=0.95)
print(mmc)


        name                 = TOPATU
        NA                   = 3.0e-01
        M                    = 3.0e+01
        eff dichroic         = 9.50e-01
        eff filter           = 1.00e+00
        eff PMT              = 3.00e-01
        Optical transmission = 2.23e-02
        Filter  transmission = 9.50e-01
        Total transmission   = 6.05e-03
        


In [13]:
dl = diffraction_limit(weakLaser, mmc) 
print(f'the difraction limit in nm is {dl/nm:5.1f}')

the difraction limit in nm is 1081.7


In [12]:
fov = FoV(x = 1 * micron, y = 1 * micron, z = 1 * nm) # assumes that laser power is quickly attenuatted with R
print(fov)
print(f'power density for laser focused in FOV  = {power_density(weakLaser,fov)/(W/cm2):5.1f} W/cm2')
I = photon_density(weakLaser, fov)
print(f'photon density for UV laser FOV (per pulse of 1 s)  = {I/(1/cm2):5.1e} ph per cm2')



        FOV; x = 1.0e-03 mm; y = 1.0e-03 mm; z = 1.0e-06 mm; area = 1.0e-06 mm2 volume = 1.0e-12 mm3
        
power density for laser focused in FOV  = 450000.0 W/cm2
photon density for UV laser FOV (per pulse of 1 s)  = 1.2e+15 ph per cm2


## Calibration fluorophore: New Fuchsin
https://en.wikipedia.org/wiki/New_fuchsine

Not to be confused with *basic* fuchsin, however, could only find Q and $\sigma$ for basic fuchsin, assume they're similar

Single absorption cross section [https://core.ac.uk/download/pdf/11534085.pdf]

$\sigma^{(L)} = (4 \pm 2) \times 10^{-17}$

QY at C = 1e-5 M [https://link.springer.com/content/pdf/10.1007/s10895-014-1369-0.pdf] (figure 6. x axis scale is a typo)

Q = 0.4 

In [13]:
fuch = Molecule(name='Fuchsin', sigma=4E-17 * cm2, Q = 0.4)  
print(fuch)
flm = fluorescence_per_molecule(fuch, I)
print('Fluorescence per molecule = {0:5.1e}'.format(flm))


        Molecule name =Fuchsin; cross section = 4.0e-17 cm2; Q =   0.4
        
Fluorescence per molecule = 1.9e-02


## Detection
ORCA Flash V3.0: [https://www.hamamatsu.com/resources/pdf/sys/SCAS0134E_C13440-20CU_tec.pdf]

In [16]:
orca = CCD(name = 'C13440-20CU', n_pixels = (2048, 2048), 
           size_pixels=(6.5*micron, 6.5*micron),  effective_area=(13.312* mm, 13.312*mm),
          linear_full_well=(3e4, 150), pixel_clock_rate=(85 * MHZ, 85*MHZ),
          dark_current=0.06, readout_noise=1.0, readout_speed=100) 

In [17]:
n_photons_reaching_CCD = flm * mmc.transmission()
print('nof photoelectrons reaching CCD = {0:5.1e}'.format(n_photons_reaching_CCD))

n_photoelectrons_CCD       = n_photons_reaching_CCD * orca.efficiency(weakLaser.lamda)
n_photoelectrons_pixel_CCD = photoelectrons_per_pixel(n_photoelectrons_CCD, orca)

print('nof photoelectrons CCD          = {0:5.1e}'.format(n_photoelectrons_CCD))
print('nof photoelectrons per pixel    = {0:5.1e}'.format(n_photoelectrons_pixel_CCD))
print('S/N per pixel                   = {0:5.1e}'.format(np.sqrt(n_photoelectrons_pixel_CCD)))

nof photoelectrons reaching CCD = 1.2e-04
nof photoelectrons CCD          = 1.1e-04
nof photoelectrons per pixel    = 2.7e-11
S/N per pixel                   = 5.2e-06


## Using a better OB

NA = 0.85

M = 60x

In [14]:
mOB = Microscope(name='TOPATU_goodOB', numerical_aperture=0.85, magnification=60, eff_filter=1, eff_dichroic=0.95)
print(mOB)


        name                 = TOPATU_goodOB
        NA                   = 8.5e-01
        M                    = 6.0e+01
        eff dichroic         = 9.50e-01
        eff filter           = 1.00e+00
        eff PMT              = 3.00e-01
        Optical transmission = 1.70e-01
        Filter  transmission = 9.50e-01
        Total transmission   = 4.60e-02
        


In [18]:
n_photons_reaching_CCD = flm * mOB.transmission()
print('nof photoelectrons reaching CCD = {0:5.1e}'.format(n_photons_reaching_CCD))

n_photoelectrons_CCD       = n_photons_reaching_CCD * orca.efficiency(weakLaser.lamda)
n_photoelectrons_pixel_CCD = photoelectrons_per_pixel(n_photoelectrons_CCD, orca)

print('nof photoelectrons CCD          = {0:5.1e}'.format(n_photoelectrons_CCD))
print('nof photoelectrons per pixel    = {0:5.1e}'.format(n_photoelectrons_pixel_CCD))
print('S/N per pixel                   = {0:5.1e}'.format(np.sqrt(n_photoelectrons_pixel_CCD)))

nof photoelectrons reaching CCD = 8.9e-04
nof photoelectrons CCD          = 8.5e-04
nof photoelectrons per pixel    = 2.0e-10
S/N per pixel                   = 1.4e-05
